In [1]:
import os
import numpy as np
import librosa
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [2]:
train_data_dir = "TrainSet"
test_data_dir = "TestSet"

train_durations = []
for root, dirs, files in os.walk(train_data_dir):
    for file in files:
        if file.endswith('.wav'):
            file_path = os.path.join(root, file)
            audio, sr = librosa.load(file_path, sr=None)
            train_durations.append(len(audio))

test_durations = []
for root, dirs, files in os.walk(test_data_dir):
    for file in files:
        if file.endswith('.wav'):
            file_path = os.path.join(root, file)
            audio, sr = librosa.load(file_path, sr=None)
            test_durations.append(len(audio))

target_duration = max(max(train_durations), max(test_durations))


In [3]:
def load_audio_with_padding(file_path):
    audio, sr = librosa.load(file_path, sr=None,)
    if len(audio) < target_duration:
        pad_width = int(target_duration - len(audio))
        audio = np.pad(audio, pad_width=((0, pad_width)), mode='constant')
    return audio, sr


In [4]:
# Function to extract MFCC features from audio file

def extract_lpc(y, num_lpc=14):
   
    
    # Extract LPC features
    lpc = librosa.core.lpc(y, order=num_lpc)
  
    
    return lpc

In [5]:
# Function to extract MFCC features from audio file
def extract_feature(file_path, num_mfcc=12,num_mel_filters=24,frame_length = 0.020  ):
    audio, sr = load_audio_with_padding(file_path)
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=num_mfcc,n_mels=num_mel_filters,n_fft=int(sr * frame_length))
  
    # Calculate Zero Crossing Rate
    zcr = librosa.feature.zero_crossing_rate(audio).sum()
    lpc=extract_lpc(audio)
    
    return np.concatenate((np.mean(mfccs,axis=1),lpc.flatten(),[zcr]))

In [6]:
# Function to load data and extract features
def load_data(data_dir, num_mfcc=12):
    labels = []
    features = []

    for root, dirs, files in os.walk(data_dir):
        for file in files:
            if file.endswith('.wav'):
                file_path = os.path.join(root, file)
                label = int(os.path.basename(root))
                labels.append(label)
                feature = extract_feature(file_path, num_mfcc=num_mfcc)
                features.append(feature)




    return features, labels

In [7]:


# Load training data
train_data_dir = "TrainSet"
X_train, y_train = load_data(train_data_dir)

# Load test data
test_data_dir = "TestSet"
X_test, y_test = load_data(test_data_dir)




In [9]:


X_train =  np.array( X_train)

X_test = np.array( X_test)

In [10]:
print(X_test.shape,X_train.shape)

(296, 28) (1900, 28)


In [11]:

# Define K values for KNN
k_values = [7, 11, 15, 20]

# Iterate over K values
for k in k_values:
    # Train KNN model
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)

    # Predict labels for test data
    y_pred = knn.predict(X_test)

    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy for K={k}: {accuracy}")


Accuracy for K=7: 0.49324324324324326
Accuracy for K=11: 0.4864864864864865
Accuracy for K=15: 0.4831081081081081
Accuracy for K=20: 0.4864864864864865


In [12]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score


# SVM with linear kernel using MFCC features
svm_linear_mfcc = SVC(kernel='linear')
svm_linear_mfcc.fit(X_train, y_train)
y_pred_linear_mfcc = svm_linear_mfcc.predict(X_test)
accuracy_linear_mfcc = accuracy_score(y_test, y_pred_linear_mfcc)
print("Accuracy of SVM with linear kernel using MFCC features:", accuracy_linear_mfcc)

# SVM with polynomial kernel using MFCC features
svm_poly_mfcc = SVC(kernel='poly')
svm_poly_mfcc.fit(X_train, y_train)
y_pred_poly_mfcc = svm_poly_mfcc.predict(X_test)
accuracy_poly_mfcc = accuracy_score(y_test, y_pred_poly_mfcc)
print("Accuracy of SVM with polynomial kernel using MFCC features:", accuracy_poly_mfcc)




Accuracy of SVM with linear kernel using MFCC features: 0.8175675675675675
Accuracy of SVM with polynomial kernel using MFCC features: 0.21621621621621623
